In [1]:
import pandas as pd 
import pyodbc 
import numpy as np
#import datetime
#from datetime import datetime
from functools import reduce
import os
from dateutil.parser import parse
import hashlib

In [2]:
start_date = '01/01/2020'
end_date = '01/01/2021'

date_range = parse(start_date).strftime("%m_%d_%Y") + '_to_' +     parse(end_date).strftime("%m_%d_%Y")
date_range

'01_01_2020_to_01_01_2021'

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ENTSQL01LSNR;'
                      'Database=EMTCQIData;'
                      'Trusted_Connection=yes;')

tat_sql= """ 
select PATIENT_FIN as pt_fin, TRACK_GROUP campus, checkin_date_time arr_dt,format(checkin_date_time, 'yyyy-MM-dd HH') hov
,  dispo_date_time dc_dt, format (dispo_date_time, 'yyyy-MM-dd HH') as hod
, format (checkin_date_time,'yyyy-MM') yov_mov
, year(checkin_date_time) yov
, month (checkin_date_time) mov
, total_ed_los los
, left(tat.PT_ACUITY,1) as esi
, case 
    when notes.result like '%99291%' then '99291'
    when notes.result like '%99285%' then '99285'
    when notes.result like '%99284%' then '99284'
    when notes.result like '%99283%' then '99283'
    when notes.result like '%99282%' then '99282'
    when notes.result like '%99281%' then '99281'
    else null
    end cpt
       
    
FROM ED_TAT_MASTER tat
    left outer join 
    (select* from
    ED_NOTES_MASTER charges
    where charges.result_title_text = 'ED Physician Charges' 
    )
    notes on tat.patient_fin = notes.pt_fin
Where Checkin_date_time BETWEEN ? and ? 
and total_ed_los > 15
order by pt_fin, cpt desc, campus desc
"""
tat = pd.read_sql(tat_sql,conn, params=[start_date,end_date])
tat = tat.drop_duplicates(subset=['pt_fin','campus'], keep='first')
tat['arr_dt'] = tat['arr_dt'].astype('datetime64')
tat['dc_dt'] = tat['dc_dt'].astype('datetime64')


#de-ID FIN
def shash(val):
    h = hashlib.sha256()
    vb = bytes(val, 'utf-8')
    h.update(vb)
    return h.digest()
tat['pt_fin'] = tat['pt_fin'].astype(str)
tat['d_fin'] = tat['pt_fin'].apply(shash)
tat=tat.drop(columns=['pt_fin'])
tat.head()

tat.head()

,campus,arr_dt,hov,dc_dt,hod,yov_mov,yov,mov,los,esi,cpt,d_fin
0,ED Tracking Group,2020-01-01 00:17:00,2020-01-01 00,2020-01-01 06:37:00,2020-01-01 06,2020-01,2020,1,379.0,None,None,b'\t\xcd\x03\xb1Xr\xb8w\xd5\x8c[\n\x19\x88O\x8...
1,ED Tracking Group,2020-01-01 02:04:00,2020-01-01 02,2020-01-01 06:36:00,2020-01-01 06,2020-01,2020,1,273.0,None,99285,b'|\xdci\xc4\x91\x0f?J\xd9\xef\x19\x9c\xcb\x0f...
3,ED Tracking Group,2020-01-01 00:10:00,2020-01-01 00,2020-01-01 03:17:00,2020-01-01 03,2020-01,2020,1,188.0,4,99284,b'\xa3\x85\xbb!EA9\x92\x16\xae6\xe8P/\xd8\xae%...
4,ED Tracking Group,2020-01-01 00:22:00,2020-01-01 00,2020-01-01 03:05:00,2020-01-01 03,2020-01,2020,1,163.0,3,99284,b'R\xf2\xd5\xf5rqV\xa7\xdcF\x9b\xfd\x03\xf8a]y...
5,EDU Tracking Group,2020-01-01 00:24:00,2020-01-01 00,2020-01-01 02:35:00,2020-01-01 02,2020-01,2020,1,132.0,5,None,b'\x0b\xf636\xb5\xb3\xfa\x9fO\xef\xdb}v\xf0\xa...


In [14]:
#create an updated hod column in order to add an hour to depart for patients that arrive and depart same hour
tat['hod1']=tat['dc_dt'].dt.strftime('%Y-%m-%d %H')

#where == if condition true, replace value in arg2 with value in arg1
tat['hod1']=np.where(tat['hov']==tat['hod'],(tat['dc_dt'] + pd.DateOffset(hours=1)).dt.strftime('%Y-%m-%d %H'),tat['hod1'])

campus_dict = {'EDU Tracking Group':'ED_SA','ED Tracking Group':'SZ'}
tat['campus'] = tat['campus'].replace(campus_dict)
tat.head()

,campus,arr_dt,hov,dc_dt,hod,yov_mov,yov,mov,los,esi,cpt,d_fin,hod1
0,SZ,2020-01-01 00:17:00,2020-01-01 00,2020-01-01 06:37:00,2020-01-01 06,2020-01,2020,1,379.0,None,None,b'\t\xcd\x03\xb1Xr\xb8w\xd5\x8c[\n\x19\x88O\x8...,2020-01-01 06
1,SZ,2020-01-01 02:04:00,2020-01-01 02,2020-01-01 06:36:00,2020-01-01 06,2020-01,2020,1,273.0,None,99285,b'|\xdci\xc4\x91\x0f?J\xd9\xef\x19\x9c\xcb\x0f...,2020-01-01 06
3,SZ,2020-01-01 00:10:00,2020-01-01 00,2020-01-01 03:17:00,2020-01-01 03,2020-01,2020,1,188.0,4,99284,b'\xa3\x85\xbb!EA9\x92\x16\xae6\xe8P/\xd8\xae%...,2020-01-01 03
4,SZ,2020-01-01 00:22:00,2020-01-01 00,2020-01-01 03:05:00,2020-01-01 03,2020-01,2020,1,163.0,3,99284,b'R\xf2\xd5\xf5rqV\xa7\xdcF\x9b\xfd\x03\xf8a]y...,2020-01-01 03
5,ED_SA,2020-01-01 00:24:00,2020-01-01 00,2020-01-01 02:35:00,2020-01-01 02,2020-01,2020,1,132.0,5,None,b'\x0b\xf636\xb5\xb3\xfa\x9fO\xef\xdb}v\xf0\xa...,2020-01-01 02


In [15]:
# create arrival number of patients by hour of visit arrival (hov)
arr_fins= pd.pivot_table(tat, values='d_fin', fill_value=0, index=['hov'],
                    columns=['campus'], aggfunc=pd.Series.nunique)
arr_fins= pd.DataFrame(arr_fins.to_records())
arr_fins

,hov,ED_SA,SZ
0,2020-01-01 00,3,7
1,2020-01-01 01,1,11
2,2020-01-01 02,4,9
3,2020-01-01 03,0,10
4,2020-01-01 04,1,6
...,...,...,...
8513,2020-12-31 19,0,10
8514,2020-12-31 20,1,2
8515,2020-12-31 21,3,6
8516,2020-12-31 22,0,6


In [16]:
#create departing number of patients for each hour using hod
dc_fins= pd.pivot_table(tat, values='d_fin', fill_value=0, index=['hod1'],
                    columns=['campus'], aggfunc=pd.Series.nunique)
dc_fins=pd.DataFrame(dc_fins.to_records())
dc_fins = dc_fins.rename(columns = {'ED_SA':'SA_dc','SZ':'SZ_dc'})
dc_fins

,hod1,SA_dc,SZ_dc
0,2020-01-01 01,2,0
1,2020-01-01 02,1,3
2,2020-01-01 03,1,9
3,2020-01-01 04,2,5
4,2020-01-01 05,2,4
...,...,...,...
8640,2021-01-01 06,0,1
8641,2021-01-01 09,0,1
8642,2021-01-01 10,0,1
8643,2021-01-01 12,0,1


In [17]:
#create a function that makes a new df with a row for each hour in both dt and str format between start and end
def every_hour(start, end):
    go = pd.to_datetime(start)
    stop = pd.to_datetime(end)
    df = pd.DataFrame()
    df['hour_dt'] = [go]
    while go < stop:
        go = go + pd.DateOffset(hours=1)
        #next_hour = go.strftime('%Y-%m-%d %H')
        df = df.append({'hour_dt':go},ignore_index = True)
    df['hour_str'] = df['hour_dt'].dt.strftime('%Y-%m-%d %H')
    return df

all_hours = every_hour(start_date,end_date)
all_hours

,hour_dt,hour_str
0,2020-01-01 00:00:00,2020-01-01 00
1,2020-01-01 01:00:00,2020-01-01 01
2,2020-01-01 02:00:00,2020-01-01 02
3,2020-01-01 03:00:00,2020-01-01 03
4,2020-01-01 04:00:00,2020-01-01 04
...,...,...
8780,2020-12-31 20:00:00,2020-12-31 20
8781,2020-12-31 21:00:00,2020-12-31 21
8782,2020-12-31 22:00:00,2020-12-31 22
8783,2020-12-31 23:00:00,2020-12-31 23


In [18]:
hourly_fins = pd.merge(left = all_hours, right = arr_fins, how='left', left_on = ['hour_str'], right_on = ['hov'], left_index=True)
hourly_fins = pd.merge(left = hourly_fins, right = dc_fins, how='left', left_on = ['hour_str'], right_on = ['hod1'], left_index=True)
hourly_fins = hourly_fins.fillna(0)
#SZ_total represents change in patient count from prior hour
hourly_fins['SZ_net'] = hourly_fins['SZ'] - hourly_fins['SZ_dc']
hourly_fins['SA_net'] = hourly_fins['ED_SA'] - hourly_fins['SA_dc']
#test_SZ represents the total patients in SZ at that hour
hourly_fins['current_census_SZ'] = hourly_fins['SZ_net'].cumsum()
hourly_fins['current_census_SA'] = hourly_fins['SA_net'].cumsum()
hourly_fins

,hour_dt,hour_str,hov,ED_SA,SZ,hod1,SA_dc,SZ_dc,SZ_net,SA_net,current_census_SZ,current_census_SA
NaN,2020-01-01 00:00:00,2020-01-01 00,2020-01-01 00,3.0,7.0,0,0.0,0.0,7.0,3.0,7.0,3.0
0.0,2020-01-01 01:00:00,2020-01-01 01,2020-01-01 01,1.0,11.0,2020-01-01 01,2.0,0.0,11.0,-1.0,18.0,2.0
1.0,2020-01-01 02:00:00,2020-01-01 02,2020-01-01 02,4.0,9.0,2020-01-01 02,1.0,3.0,6.0,3.0,24.0,5.0
2.0,2020-01-01 03:00:00,2020-01-01 03,2020-01-01 03,0.0,10.0,2020-01-01 03,1.0,9.0,1.0,-1.0,25.0,4.0
3.0,2020-01-01 04:00:00,2020-01-01 04,2020-01-01 04,1.0,6.0,2020-01-01 04,2.0,5.0,1.0,-1.0,26.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8630.0,2020-12-31 20:00:00,2020-12-31 20,2020-12-31 20,1.0,2.0,2020-12-31 20,1.0,8.0,-6.0,0.0,26.0,4.0
8631.0,2020-12-31 21:00:00,2020-12-31 21,2020-12-31 21,3.0,6.0,2020-12-31 21,3.0,6.0,0.0,0.0,26.0,4.0
8632.0,2020-12-31 22:00:00,2020-12-31 22,2020-12-31 22,0.0,6.0,2020-12-31 22,2.0,5.0,1.0,-2.0,27.0,2.0
8633.0,2020-12-31 23:00:00,2020-12-31 23,2020-12-31 23,1.0,3.0,2020-12-31 23,0.0,2.0,1.0,1.0,28.0,3.0
